In [ ]:
#installing the essentials
!pip install cmake
!pip install dlib
!pip install face_recognition
!pip install wheel
!pip install opencv-python

#mounting the drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import face_recognition
import os

In [ ]:
#importing the dataset
path = 'dataset path'
images = []
classNames = []
mylist = os.listdir(path)
print(mylist)
for cl in mylist:
     curImg = cv2.imread(f'{path}/{cl}')
     images.append(curImg)
     classNames.append(os.path.splitext(cl)[0])
print(classNames)

In [ ]:
    #finding the encoding of each face
def findEncodings(images):
    encodeList = []
    for img in images:
      img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
      encode = face_recognition.face_encodings(img)[0]
      encodeList.append(encode)
    return encodeList

encodeListKnown = findEncodings(images)
print(len(encodeListKnown))  

In [ ]:
cap = cv2.VideoCapture(0)
try:
  while True:
    success, img = cap.read()
    try:
      imgS = cv2.resize(img,(0,0),None,0.25,0.25)
      imgS = cv2.cvtColor(imgS,cv2.COLOR_BGR2RGB)
    except:
      break;
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)

    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        print(faceDis)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
          name = classNames[matchIndex].upper()
          print(name)
          y1,x2,y2,x1 = faceLoc
          y1,x2,y2,x1 = y1*4, x2*4, y2*4, x1*4
          cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
          cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
          cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
          markAttendence(name)

    cv2.imshow('WebCam',img)
    cv2.waitKey(1)
except KeyboardInterrupt:
   exit(0)


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))